In [2]:
import matplotlib
#%matplotlib tk
%matplotlib inline
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [2]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
#fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = cohort.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

#
cohort.ethogram = ethogram

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       2     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1           1       2     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2           2       2     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3           3       2     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4           4       

In [3]:
#################################################
########### COMPUTE HUDDLE COMPOSITION ##########
#################################################

#
cohort.video_frame_rate = 24
cohort.parallel_flag = True

# set the flags for the huddle corrections
cohort.fix_track_flag = True
cohort.interpolate_flag = True
cohort.load_huddle_tracks()

# # this just loads the _spine.npy files;
cohort.use_nohuddle = True
cohort.load_feature_tracks()

# # this computes mode-based occupancy in the nest with 1min bins
cohort.median_filter_width = cohort.video_frame_rate*5+1 # 1: there is no smotthing; n x frame rate gives you n seconds of somothing (always add 1 to get odd number)
#cohort.n_frames_per_bin = cohort.video_frame_rate  # single second resolution
cohort.n_frames_per_bin = cohort.video_frame_rate #*60  # single second resolution
cohort.n_cores = 24
cohort.compute_huddle_composition()
print ("# of huddle composition arrays: ", len(cohort.huddle_comps_binned), cohort.huddle_comps_binned[0].shape)


 # of files:  991 example:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/huddles/2020_07_20_09_53_59_813912_compressed_Day_huddle_spine_fixed_interpolated.npy
# of feature tracks:  991


1001it [02:51,  5.85it/s]                                                              


# of huddle composition arrays:  991 (6, 1201)


In [8]:
#################################################
########### DISPLAY HUDDLE COMPOSITION ##########
#################################################
%matplotlib qt

#
cohort.forward_padding = 20 # padding in percent
cohort.generate_huddle_composition_ethograms()

#
cohort.show_huddle_composition_ethogram_all_animals()

#
#cohort.show_huddle_composition_ethogram_full_huddle()

#
animal_ids = np.arange(6)
# cohort.show_huddle_composition_ethogram_specific_animals(animal_ids)


size of img:  (86400, 112)  size of flatten image:  (1382400, 6)


100%|██████████████████████████████████████████████| 991/991 [00:00<00:00, 2432.74it/s]


In [9]:
print (cohort.huddle_ethogram.shape)

(86400, 112)


In [11]:
####################################################
########### SELECT DAY OF ETHOGRAM TO ANALYZ #######
####################################################

# NEED TO run previous cell to compute "huddle_ethogram" values

#
ethogram_flat = []

for k in range(0,cohort.huddle_ethogram.shape[1],7):
    ethogram_flat.append(cohort.huddle_ethogram[:,k:k+6])
ethogram_flat = np.vstack(ethogram_flat).T
print (ethogram_flat.shape)

# here P15 is day 0 for clarity
day_start = 0
day_end = 16

#
times = np.arange(day_start*24*60*60,
                  day_end*24*60*60)
print ("# animals, # seconds: ", ethogram_flat.shape)

#
ethogram_flat_selected = ethogram_flat[:,times].copy()
print (ethogram_flat_selected.shape)




(6, 1382400)
# animals, # seconds:  (6, 1382400)
(6, 1382400)


In [12]:
####################################################
########### COMPUTE PAIR WISE SCATTER PLOTS ########
####################################################

%matplotlib qt
from tqdm import trange

plt.figure(figsize=(6,6))
clrs = ['red','blue','green','black','purple','orange']
clrs = ['#cb3311', '#4477aa', '#ccbb44', '#228833', '#ee7733', '#66ccee']
markers = ["," , "o" , "v" , "^" , "<", ">"]

#
follow_window = 10
xlim=0
ratios=[]
for k in trange(6):
# for k in [0]:
    # for p in range(k+1,6,1):
    for p in range(6):
    # for p in [1]:
        if k==p:
            continue
        t1 = ethogram_flat_selected[k].copy()
        idx1 = np.where(t1>0)[0]
        t1[idx1] = 1
        t2 = ethogram_flat_selected[p].copy()
        idx2 = np.where(t2>0)[0]
        t2[idx2] = 1

        # find order of events
        a1 = 0     #the number of times animal 1 precedes animal 2
        a2 = 0     # same but opposite order

        #
        ctr = 0
        while ctr<idx1.shape[0]:
        #for i in idx1:

            # check to see if there is a switch from 0
            if t2[idx1[ctr]]==0:
                # to a 1 in the next window
                if np.sum(t2[idx1[ctr]:idx1[ctr]+follow_window])>0:
                    a1+=1

                    # also advance the ctr value at least 10 sec
                    ctr0 = ctr
                    try:
                        while idx1[ctr+1]<(idx1[ctr0]+follow_window) and (ctr<idx1.shape[0]-2):
                            ctr+=1
                            continue
                    except:
                        pass
            ctr+=1

        #
        ctr = 0
        while ctr<idx2.shape[0]:
        #for i in idx1:

            # check to see if there is a switch from 0
            if t1[idx2[ctr]]==0:
                # to a 1 in the next window
                if np.sum(t1[idx2[ctr]:idx2[ctr]+follow_window])>0:
                    a2+=1

                    # also advance the ctr value at least 10 sec
                    ctr0 = ctr
                    try:
                        while idx2[ctr+1]<(idx2[ctr0]+follow_window) and (ctr<idx2.shape[0]-2):
                            ctr+=1
                            continue
                    except:
                        pass
            ctr+=1
        xlim = np.max((xlim,a1,a2))
        plt.scatter(a1,
                    a2,
                    #marker=markers[k],
                    marker=markers[p],
                    s=100,
                    #c=clrs[p],
                    c=clrs[k],

                    label=str(k)+'->'+str(p))


        # print ("animal1 : ", k, ", animal2: ", p, "# of leading behaviours: ", a1)
        # print ("animal1 : ", k, ", animal2: ", p, "# of following behaviours: ", a2)
        ratio = a1/(a1+a2)
        # print ("ratio: ", ratio)
        ratios.append([a1,a2,ratio])
        #print ("a2: ", a2)
#
plt.plot([0,xlim*1.25],[0,xlim*1.25],'--',c='black')
plt.legend(fontsize=8, ncol=6)
plt.title("# of behavior sequences (1sec resulution; 10sec follow window))")
#plt.xlim(0,700) 
# plt.xlim(left=0)
#plt.ylim(0,700) 
# plt.ylim(bottom=0)
plt.title("start day: "+str(day_start) + ",  day end: "+str(day_end))
plt.xlabel("Animal 1 precedes animal 2")
plt.ylabel("Animal 2 precedes animal 1")
plt.show()
# plt.savefig('/home/cat/Downloads/P26-.png')


100%|████████████████████████████████████████████████████| 6/6 [02:47<00:00, 27.95s/it]


In [14]:
#########################################################################################
########### COMPUTE PAIR WISE SCATTER PLOTS - FOR SINGLE ANIMAL OVER DEVELOPMENT ########
#########################################################################################

plt.figure(figsize=(6,6))
clrs = ['red','blue','green','black','purple','orange']
clrs = ['#cb3311', '#4477aa', '#ccbb44', '#228833', '#ee7733', '#66ccee']
markers = ["," , "o" , "v" , "^" , "<", ">"]

#
follow_window = 10
xlim=0
ratios = np.empty((16,6,6))
ratios[:] = np.nan

day_starts = np.arange(17)
day_ends = np.arange(1,17,1)


#
for d in trange(16):
    day_start = day_starts[d]
    day_end = day_ends[d]
    #
    times = np.arange(day_start*24*60*60,
                      day_end*24*60*60)
    #print ("times: ", times)

    #
    ethogram_flat_selected = ethogram_flat[:,times].copy()

    #for k in trange(6):
    for k in [1]:
        #for p in range(k+1,6,1):
        for p in range(6):
        # for p in [3]:
            if k==p:
                continue
            t1 = ethogram_flat_selected[k].copy()
            idx1 = np.where(t1>0)[0]
            t1[idx1] = 1
            t2 = ethogram_flat_selected[p].copy()
            idx2 = np.where(t2>0)[0]
            t2[idx2] = 1

            # find order of events
            a1 = 0     #the number of times animal 1 precedes animal 2
            a2 = 0     # same but opposite order

            #
            ctr = 0
            while ctr<idx1.shape[0]:
            #for i in idx1:

                # check to see if there is a switch from 0
                if t2[idx1[ctr]]==0:
                    # to a 1 in the next window
                    if np.nansum(t2[idx1[ctr]:idx1[ctr]+follow_window])>0:
                        a1+=1

                        # also advance the ctr value at least 10 sec
                        ctr0 = ctr
                        try:
                            while idx1[ctr+1]<(idx1[ctr0]+follow_window) and (ctr<idx1.shape[0]-2):
                                ctr+=1
                                continue
                        except:
                            pass
                ctr+=1

            #
            ctr = 0
            while ctr<idx2.shape[0]:
            #for i in idx1:

                # check to see if there is a switch from 0
                if t1[idx2[ctr]]==0:
                    # to a 1 in the next window
                    if np.nansum(t1[idx2[ctr]:idx2[ctr]+follow_window])>0:
                        a2+=1

                        # also advance the ctr value at least 10 sec
                        ctr0 = ctr
                        try:
                            while idx2[ctr+1]<(idx2[ctr0]+follow_window) and (ctr<idx2.shape[0]-2):
                                ctr+=1
                                continue
                        except:
                            pass
                ctr+=1
            xlim = np.max((xlim,a1,a2))



            # print ("animal1 : ", k, ", animal2: ", p, "# of leading behaviours: ", a1)
            # print ("animal1 : ", k, ", animal2: ", p, "# of following behaviours: ", a2)
            ratio = a1/(a1+a2)
            # print ("ratio: ", ratio)
            # ratios.append(ratio)
            #print (ratios)
            #print (d,p,k,ratio)
            ratios[d,p,k] = ratio
            #print ("a2: ", a2)
    #
# plt.plot([0,xlim*1.25],[0,xlim*1.25],'--',c='black')
# plt.legend(fontsize=8, ncol=6)
# plt.title("# of behavior sequences (1sec resulution; 10sec follow window))")
# plt.xlim(0,700) 
# # plt.xlim(left=0)
# plt.ylim(0,700) 
# # plt.ylim(bottom=0)
# plt.title("start day: "+str(day_start) + ",  day end: "+str(day_end))
# plt.xlabel("Animal 1 precedes animal 2")
# plt.ylabel("Animal 2 precedes animal 1")
# plt.show()
# plt.savefig('/home/cat/Downloads/P17.png')



100%|██████████████████████████████████████████████████| 16/16 [00:29<00:00,  1.87s/it]


In [18]:
###########################################################################
########### GET ALL the times of the starts of following behaviors ########
###########################################################################

clrs = ['red','blue','green','black','purple','orange']
clrs = ['#cb3311', '#4477aa', '#ccbb44', '#228833', '#ee7733', '#66ccee']
markers = ["," , "o" , "v" , "^" , "<", ">"]

#
follow_window = 10
xlim=0
ratios = np.empty((16,6,6))
ratios[:] = np.nan

day_starts = np.arange(17)
day_ends = np.arange(1,17,1)

#
animal1 = 0
animal2 = 1

# computing times of when animal 1 follows animal2
#ethogram_flat_selected = ethogram_flat[:,times].copy()

# get times of animal 1
t1 = ethogram_flat[animal1].copy()
idx1 = np.where(t1>0)[0]
t1[idx1] = 1

# get times of animal 2
t2 = ethogram_flat[animal2].copy()
idx2 = np.where(t2>0)[0]
t2[idx2] = 1

# find order of events
a1 = 0     #the number of times animal 1 precedes animal 2

#
ctr = 0
times = []
while ctr<idx1.shape[0]:
#for i in idx1:

    # check to see if there is a switch from 0
    if t2[idx1[ctr]]==0:
        # to a 1 in the next window
        if np.sum(t2[idx1[ctr]:idx1[ctr]+follow_window])>0:
            a1+=1
            times.append(idx1[ctr])

            # also advance the ctr value at least 10 sec
            ctr0 = ctr
            try:
                while idx1[ctr+1]<(idx1[ctr0]+follow_window) and (ctr<idx1.shape[0]-2):
                    ctr+=1
                    continue
            except:
                pass
    ctr+=1

print ("Done: ", a1)
times = np.array(times)
print (times)

plt.figure()
x = np.arange(ethogram_flat.shape[1])/60/60
times_absolute = np.zeros(x.shape[0])
times_absolute[times]=1

#import scipy
#times_absolute_smooth = scipy.signal.savgol_filter(times_absolute,101,2)
plt.plot(x,times_absolute)

plt.show()


print ("Sequence array is the times_absolute aray: ", times_absolute.shape)
print ("times absolute: ", times_absolute)
np.save('/home/cat/Downloads/timeseries.npy', times_absolute)


Done:  3913
[  36594   37160   37655 ... 1332963 1332973 1333332]
Sequence array is the times_absolute aray:  (1382400,)
times absolute:  [0. 0. 0. ... 0. 0. 0.]


In [244]:
np.save('/home/cat/Downloads/leader-follower/ratios.npy', ratios)

ratios[:,1,0]

array([0.48427673, 0.49908592, 0.46300211, 0.58646617, 0.43883495,
       0.51442308, 0.39106145, 0.48926014, 0.50680787, 0.43835616,
       0.45875543, 0.40865385, 0.45537341, 0.4939577 , 0.44193548,
       0.54883721])

In [4]:
ratios = np.load('/home/cat/Downloads/leader-follower/ratios.npy')

In [ ]:
test = np.mean(np.mean(ratios[:,2:,0:2], axis = 2), axis = 1)
test.shape

sdt = np.std(np.std(ratios[:,2:,0:2], axis = 2), axis = 1)
sdt.shape

In [1]:
test = np.mean(np.mean(ratios[:,0:2,2:], axis = 2), axis = 1)
test.shape

sdt = np.std(np.std(ratios[:,0:2,2:], axis = 2), axis = 1)
sdt.shape

NameError: name 'np' is not defined

In [35]:
test = np.mean(ratios[:,2:,1], axis = 1)
test.shape

sdt = np.std(ratios[:,2:,1], axis = 1)
sdt.shape

(16,)

In [36]:
%matplotlib qt

print (ratios.shape)

plt.figure()

plt.plot(test, color ='red', linewidth = 3, label = 'all pups')
    
plt.xlim(0,15)
plt.xticks(ticks = np.arange(0,16), labels = np.arange(15,31))
plt.xlabel("Postnatal day")

plt.ylim(0,1)
plt.yticks(np.arange(0,1.1,0.1))
plt.ylabel("Leader / Follower ratio")

plt.hlines(0.5,0,15, color = 'black', linestyle = 'dashdot')

plt.legend()

plt.title("Male  vs All pups")

plt.fill_between(np.arange(0,16), np.array([t-s for t,s in zip(test,sdt)]), np.array([t+s for t,s in zip(test,sdt)]))

plt.show()


(16, 6, 6)


Gtk-Message: 13:22:14.692: GtkDialog mapped without a transient parent. This is discouraged.

(python:5649): Gtk-WARNING **: 13:22:14.971: Failed to measure available space: The specified location is not supported


In [9]:
%matplotlib qt

print (ratios.shape)

animals = ['female', 'male', 'pup1', 'pup2', 'pup3', 'pup4']

plt.figure()

clrs = ['#cb3311', '#4477aa', '#ccbb44', '#228833', '#ee7733', '#66ccee']

for i, a in enumerate(clrs):
    
    plt.plot(ratios[:,i,1], color = a, linewidth = 3, label = animals[i])
    
    plt.xlim(0,15)
    plt.xticks(ticks = np.arange(0,16), labels = np.arange(15,31))
    plt.xlabel("Postnatal day")
    
    plt.ylim(0,1)
    plt.yticks(np.arange(0,1.1,0.1))
    plt.ylabel("Leader / Follower ratio")
    
    plt.hlines(0.5,0,15, color = 'black', linestyle = 'dashdot')
    
    plt.legend()
    
    plt.title("Male")

plt.show()


(16, 6, 6)


Gtk-Message: 12:23:48.983: GtkDialog mapped without a transient parent. This is discouraged.

(python:5649): Gtk-WARNING **: 12:23:49.215: Failed to measure available space: The specified location is not supported
